In [5]:
import sys
sys.executable

'/home/newman/anaconda3/bin/python'

In [3]:
from jupyter_core.paths import jupyter_data_dir
print(jupyter_data_dir())

/home/newman/.local/share/jupyter


In [1]:
from main import make_model
import keras
from keras import backend as K
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
#remember to clear session using K.clear_session()

ModuleNotFoundError: No module named 'tensorflow'

Drawn from keras-team examples simple mnist_cnn.py

In [9]:
model_spec = {
  "__info__": "This is a JSON file used for testing the model creation process",
  "model_name": "Test-v0",
  "num_layers": 1,
  "loss": "mse",
  "optimizer": "rmsprop",
  "input_layer": {
    "dim": "(6, 6, 6)"
  },
  "layer_0": {
    "layer": "DenseLyr",
    "units": 128,
    "activation": "relu"
  }
}

In [22]:
mnist_cnn = {
    "__info__": "Mnist CNN model from keras-team examples",
    "model_name": "simple CNN",
    "num_layers": 7,
    "loss" : "categorial_crossentropy",
    "optimizer": "Adadelta",
    "input_layer": {
        "dim": "(28, 28, 1)"
    },
    "layer_0": {
        "layer": "Conv2DLyr",
        "filters": 32,
        "kernel_size": (3,3),
        "activation": 'relu',
    },
    "layer_1": {
        "layer": "Conv2DLyr",
        "filters": 64,
        "kernel_size": (3,3),
        "activation":'relu'
    },
    "layer_2": {
        "layer": "MaxPooling2DLyr",
        "pool_size" : (2,2)
    },
    "layer_3": {
        "layer": "DropoutLyr",
        "rate": 0.25
    },
    "layer_4": {
        "layer":"FlattenLyr"
    },
    "layer_5": {
        "layer": "DenseLyr",
        "units": 128,
        "activation":'relu'
    },
    "layer_6": {
        "layer": "DropoutLyr",
        "rate": 0.5
    },
    "layer_7": {
        "layer": "DenseLyr",
        "units": "num_classes",
        "activation":"softmax"
    }
}

## Implementing Baseline CNN Architecture (from SpecDict)

In [3]:
spec_model = make_model(mnist_cnn)

AttributeError: 'Model' object has no attribute 'shape'

In [3]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

In [12]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 3s 0us/step


In [13]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [15]:
input_shape

(28, 28, 1)

In [16]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [17]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## Same CNN Architecture (from pure Keras)

In [30]:
#use modelgraph to create model instead
model_graph = main.make_model(mnist_cnn)


KeyError: 'Conv2DLyr'

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [10]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 98s 2ms/step - loss: 0.2616 - acc: 0.9186 - val_loss: 0.0624 - val_acc: 0.9797
Epoch 2/12
  384/60000 [..............................] - ETA: 1:35 - loss: 0.1780 - acc: 0.9609

KeyboardInterrupt: 

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])